In [1]:
%load_ext autoreload
%autoreload 2
#%matplotlib nbagg
from Object_detection_features_old import *
import gym
from gym import wrappers

% matplotlib inline

In [2]:
env = gym.make('Skiing-v0')
odf = ObjectDetectionFeatures(env)

INFO:gym.envs.registration:Making new env: Skiing-v0
[2017-02-07 15:43:19,801] Making new env: Skiing-v0


In [3]:
from skimage.transform import resize
from skimage.color import rgb2gray

In [4]:
import warnings
warnings.filterwarnings('ignore')

## First, let's collect some samples

In [5]:
max_observations = 20000
observations = []
render = False
count = 0
output_shape = (60, 60)

env_name = 'Skiing-v0'
env = gym.make(env_name)

while True:
    if len(observations) >= max_observations: break
    s = env.reset()
    if count % 10 == 0:
        observation = (resize(odf.get_simple_image(env.ale.getScreenGrayscale()[:, :, 0]), 
                              output_shape, order=0) * 255).astype('uint8')
        observations.append(observation)
    count += 1
    done = False

    while not done:
        if render: env.render()
        if len(observations) >= max_observations: break
        a = env.action_space.sample()
        s, r, done, _ = env.step(a)
        if count % 10 == 0:
            observation = (resize(odf.get_simple_image(env.ale.getScreenGrayscale()[:, :, 0]), 
                                  output_shape, order=0) * 255).astype('uint8')
            observations.append(observation)
            if not len(observations) % 1000:
                print(len(observations))
        a = env.action_space.sample()
        count += 1
        
env.close()

INFO:gym.envs.registration:Making new env: Skiing-v0
[2017-02-07 15:44:35,167] Making new env: Skiing-v0


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000


In [28]:
observations = np.array(observations)
observations = observations.reshape(20000, 1, 60, 60)

## Train an autoencoder

In [9]:
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, UpSampling2D, Reshape, Flatten
from keras.models import Model
border_mode = 'same'

input_img = Input(shape=observations.shape[1:])

x = Convolution2D(16, 8, 8, activation='relu', border_mode=border_mode)(input_img)
x = MaxPooling2D((2, 2), border_mode=border_mode)(x)
x = Convolution2D(32, 4, 4, activation='relu', border_mode=border_mode)(x)
x = MaxPooling2D((2, 2), border_mode=border_mode)(x)
filters_shape = x.get_shape()
flattened = Flatten()(x)
flat_shape = flattened.get_shape()
encoded = Dense(64, activation='relu')(flattened)

x = Dense(int(flat_shape[1]), activation='relu')(encoded)
x = Reshape(tuple([int(shp) for shp in filters_shape[1:]]))(x)
x = Convolution2D(32, 4, 4, activation='relu', border_mode=border_mode)(x)
x = UpSampling2D((2, 2))(x)
x = Convolution2D(16, 8, 8, activation='relu', border_mode=border_mode)(x)
x = UpSampling2D((2, 2))(x)
x = Convolution2D(1, 3, 3, activation='relu', border_mode=border_mode)(x)

decoded = x

encoder = Model(input_img, encoded)
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='mse')

Using TensorFlow backend.


In [10]:
autoencoder.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 1, 60, 60)     0                                            
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 16, 60, 60)    1040        input_1[0][0]                    
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 16, 30, 30)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 32, 30, 30)    8224        maxpooling2d_1[0][0]             
___________________________________________________________________________________________

In [11]:
import keras
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(float(logs.get('loss')))
    
    def on_epoch_end(self, epoch, logs={}):
        self.val_losses.append(logs.get('val_loss'))


In [12]:
history = LossHistory()
autoencoder.fit(observations, observations,
                nb_epoch=40,
                batch_size=32,
                shuffle=True,
                callbacks=[history])

Epoch 1/40
20000/20000 [==============================] - 310s - loss: 466.0637   
Epoch 2/40
20000/20000 [==============================] - 346s - loss: 177.0336   
Epoch 3/40
20000/20000 [==============================] - 350s - loss: 132.9763   
Epoch 4/40
20000/20000 [==============================] - 356s - loss: 111.7421   
Epoch 5/40
20000/20000 [==============================] - 359s - loss: 96.2776   
Epoch 6/40
20000/20000 [==============================] - 356s - loss: 84.9050   
Epoch 7/40
20000/20000 [==============================] - 354s - loss: 77.5212   
Epoch 8/40
20000/20000 [==============================] - 357s - loss: 70.2496   
Epoch 9/40
20000/20000 [==============================] - 357s - loss: 66.2044   
Epoch 10/40
20000/20000 [==============================] - 361s - loss: 58.1283   
Epoch 11/40
20000/20000 [==============================] - 363s - loss: 54.8720   
Epoch 12/40
20000/20000 [==============================] - 363s - loss: 52.6134   
Epoch 13/

## And save our results

In [ ]:
import os
if not 'data' in os.listdir('.'):
    os.mkdir('./data/')
sample_features = encoder.predict(observations)
np.savez('./data/sample_features_20k.npz', sample_features)

In [27]:
autoencoder.save_weights('./data/Autoencoder_21_01.h5')
autoencoder.to_json()
import json
with open('./data/Autoencoder_21_01.txt', 'w') as outfile:
    json.dump(autoencoder.to_json(), outfile)

In [28]:
encoder.save_weights('./data/Encoder_21_01.h5')
autoencoder.to_json()
import json
with open('./data/Encoder_21_01.txt', 'w') as outfile:
    json.dump(encoder.to_json(), outfile)